In [ ]:
"""
Inspired by

- https://sebastianraschka.com/blog/2023/llm-mixed-precision-copy.html
- https://medium.com/@manindersingh120996/practical-guide-to-fine-tune-llms-with-lora-c835a99d7593
- https://cameronrwolfe.substack.com/p/basics-of-reinforcement-learning
- https://themeansquare.medium.com/reinforcement-learning-from-human-feedback-rlhf-a-practical-guide-with-pytorch-examples-139cee11fc76
- https://medium.com/analytics-vidhya/coding-ppo-from-scratch-with-pytorch-part-1-4-613dfc1b14c8 (Still not implemented yet)
- https://medium.com/@gupta.aman/learning-ppo-by-writing-your-own-training-loop-6907e23162d9
- https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/

###  I dedicated all of my sanity to this notebook ;=;  ###
                                              Dungeon master
"""

'\nInspired by\n\n- https://sebastianraschka.com/blog/2023/llm-mixed-precision-copy.html\n- https://medium.com/@manindersingh120996/practical-guide-to-fine-tune-llms-with-lora-c835a99d7593\n- https://cameronrwolfe.substack.com/p/basics-of-reinforcement-learning\n- https://themeansquare.medium.com/reinforcement-learning-from-human-feedback-rlhf-a-practical-guide-with-pytorch-examples-139cee11fc76\n- https://medium.com/analytics-vidhya/coding-ppo-from-scratch-with-pytorch-part-1-4-613dfc1b14c8 (Still not implemented yet)\n- https://medium.com/@gupta.aman/learning-ppo-by-writing-your-own-training-loop-6907e23162d9\n- https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/\n\n###  I dedicated all of my sanity to this notebook ;=;  ###\n                                              Dungeon master\n'

In [ ]:
!pip install torch transformers datasets accelerate bitsandbytes
!pip install git+https://github.com/Lightning-AI/lightning@master

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import os
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F

from transformers import AutoTokenizer
from lightning import Fabric

# import pytorch_lightning as pl
import warnings
warnings.simplefilter("ignore", UserWarning)
from torch.optim import *
from tqdm.notebook import tqdm
import gc
from peft import LoraConfig, get_peft_model, PeftModelForCausalLM

In [ ]:
import os
import pandas as pd
import re


In [ ]:
if not os.path.exists("/content/version3.zip"):
  !gdown --id 1VLBOBFV6HxI4Ke2fBQWe4X3x4C4hp9TK

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1VLBOBFV6HxI4Ke2fBQWe4X3x4C4hp9TK
From (redirected): https://drive.google.com/uc?id=1VLBOBFV6HxI4Ke2fBQWe4X3x4C4hp9TK&confirm=t&uuid=55cea3a7-5fc8-4204-9693-f9bf32ffbc41
To: /content/version3.zip
100% 32.9M/32.9M [00:00<00:00, 38.5MB/s]


In [ ]:
if not os.path.exists("/content/out"):
  !unzip /content/version3.zip

Archive:  /content/version3.zip
   creating: cleaned_data/
  inflating: __MACOSX/._cleaned_data  
  inflating: cleaned_data/C1E093.json  
  inflating: __MACOSX/cleaned_data/._C1E093.json  
  inflating: cleaned_data/C2E022.json  
  inflating: __MACOSX/cleaned_data/._C2E022.json  
  inflating: cleaned_data/C2E034.json  
  inflating: __MACOSX/cleaned_data/._C2E034.json  
  inflating: cleaned_data/C1E085.json  
  inflating: __MACOSX/cleaned_data/._C1E085.json  
  inflating: cleaned_data/C1E011.json  
  inflating: __MACOSX/cleaned_data/._C1E011.json  
  inflating: cleaned_data/C2E018.json  
  inflating: __MACOSX/cleaned_data/._C2E018.json  
  inflating: cleaned_data/C1E046.json  
  inflating: __MACOSX/cleaned_data/._C1E046.json  
  inflating: cleaned_data/C1E103.json  
  inflating: __MACOSX/cleaned_data/._C1E103.json  
  inflating: cleaned_data/C1E050.json  
  inflating: __MACOSX/cleaned_data/._C1E050.json  
  inflating: cleaned_data/C1E115.json  
  inflating: __MACOSX/cleaned_data/._C1E115

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLP/project/llama')

Mounted at /content/drive


In [ ]:
dfs = []

directory = '/content/out'
for entry in os.scandir(directory):
  if entry.is_file():
    path = entry.path
    df = pd.read_csv(path)
    dfs.append(df)

In [ ]:
df = pd.concat(dfs)
df = df.dropna()
df.shape

(398523, 3)

In [ ]:
df = df[(df['dm'] != '<|begin_of_text|><|end_of_text|>')].reset_index(drop=True)
df

,fileID,before_utterances,dm
0,C1E060,<PLAYER38> She's got the hipster glasses to pr...,"Oh boy. That's a lot, that's a lot of people."
1,C1E060,"<PLAYER38> And the next guy, the next guy, get...","All righty, let me shore up my positioning her..."
2,C1E060,<PLAYER38> Tonight we have one of those names ...,"(shushing noises) Anyway, welcome everybody to..."
3,C1E060,"<DM> Oh boy. That's a lot, that's a lot of peo...","Welcome to Critical Role, where a bunch of us ..."
4,C1E060,"<PLAYER2> Oh my gosh.\n<DM> All righty, let me...",The current Loot Crate theme for July is Futur...
...,...,...,...
111095,C1E042,<PLAYER7> I'm not fighting him!\n<DM> All righ...,So you're going to the Raven's Crest. Got it.
111096,C1E042,<PLAYER2> I'm going to stock up on a bunch of ...,"All right, cool. We'll pick it up next week at..."
111097,C1E042,"<DM> All right, cool. We'll pick it up next we...",Damn.
111098,C1E042,"<PLAYER8> If you love it, you have to--\n<PLAY...",That's horrible.


In [ ]:
train_ids = pd.read_csv("/content/train_files", header=None).squeeze("columns")
val_ids = pd.read_csv("/content/val_files", header=None).squeeze("columns")
test_ids = pd.read_csv("/content/test_files", header=None).squeeze("columns")

In [ ]:
train_df = df[df["fileID"].isin(train_ids)]
val_df = df[df["fileID"].isin(val_ids)]
test_df = df[df["fileID"].isin(test_ids)]

In [ ]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(87988, 3)
(11570, 3)
(11542, 3)


# Prepare Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from google.colab import userdata
import difflib

In [ ]:
from huggingface_hub import login
login(userdata.get('huggingFace'))

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Automatically map layers across GPUs if available
    load_in_4bit=True,   # Enables memory-efficient training
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,padding_side='left'
)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
class TeacherModel():
  def __init__(self):
    self.model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    self.model = AutoModelForCausalLM.from_pretrained(
      self.model_name,
      device_map="auto",  # Automatically map layers across GPUs if available
      load_in_4bit=True,   # Enables memory-efficient training Meta-Llama-3-8B-Instruct meta-llama/Llama-3.1-8B-Instruct
      torch_dtype=torch.bfloat16,
    )
    self.model.eval()
    for param in self.model.parameters():
      param.requires_grad = False
    self.model.gradient_checkpointing_enable()
    self.model.enable_input_require_grads()
    self.tokenizer = tokenizer

  def score(self, player_prompt, dm_response):
    rewardPrompt = [(
    "<|begin_of_text|>\n"
    "<|start_header_id|>system<|end_header_id|>\n"
    "You are a specialized evaluation system called 'The DM Judge'. Your only task is to rate Dungeon Master responses in tabletop roleplaying games. You MUST output EXACTLY ONE RATING from this list: Very Bad, Bad, Neutral, Good, or Very Good. DO NOT output any explanation, reasoning, additional text, quotation marks, or numbers.\n"
    "\n"
    "Please evaluate this DM response based on these criteria:\n"
    "  - Relevance\n"
    "  - Coherence & Clarity\n"
    "  - Creativity & Flavor\n"
    "  - Rules & Logic\n"
    "  - Engagement & Agency\n"
    "\n"
    "Rating definitions:\n"
    "  - Very Bad: Fails almost all criteria, including contextual relevance.\n"
    "  - Bad: Superficial, many issues.\n"
    "  - Neutral: Adequate but unremarkable.\n"
    "  - Good: Solid response; few nitpicks.\n"
    "  - Very Good: Excellent world-building, clarity, and engagement.\n"
    "\n"
    "Core D&D 5e Rules Summary (Updated for 2024 Revision):\n"
    "\n"
    "Combat Rules:\n"
    "  - Initiative: Determines turn order in combat via a Dexterity check (d20 + Dexterity modifier).\n"
    "  - Actions: On their turn, a character can perform one action (e.g., Attack, Cast a Spell, Dash), one bonus action (if available), and move up to their speed.\n"
    "  - Reactions: Special actions triggered outside a character's turn, such as opportunity attacks.\n"
    "  - Movement: Standard movement is affected by terrain and conditions; difficult terrain costs extra movement, and certain actions like standing from prone consume movement.\n"
    "  - Attack Rolls: Roll a d20, add relevant modifiers; if the total equals or exceeds the target's Armor Class (AC), the attack hits.\n"
    "  - Damage and Healing: Damage reduces hit points (HP); healing can be done via spells, potions, or rests.\n"
    "  - Death and Dying: At 0 HP, a character falls unconscious and must make death saving throws to avoid death.\n"
    "\n"
    "Spellcasting:\n"
    "  - Spell Slots: Spells consume slots of their level or higher; slots are replenished after a long rest.\n"
    "  - Spell Components: Spells may require verbal (V), somatic (S), and/or material (M) components.\n"
    "  - Concentration: Some spells require concentration; a caster can concentrate on only one spell at a time and must make Constitution saving throws to maintain concentration when taking damage.\n"
    "  - Cantrips: Zero-level spells that can be cast at will without expending spell slots.\n"
    "\n"
    "Resting and Recovery:\n"
    "  - Short Rest: A period of at least 1 hour where characters can spend Hit Dice to regain HP.\n"
    "  - Long Rest: A period of at least 8 hours, restoring all HP, expended spell slots, and half of the character's total Hit Dice.\n"
    "\n"
    "Advantage and Disadvantage:\n"
    "  - Advantage: Roll two d20s and take the higher result.\n"
    "  - Disadvantage: Roll two d20s and take the lower result.\n"
    "\n"
    "DM Conduct Guidelines:\n"
    "  - Role and Tone: The DM should maintain an impartial narrator role, describe environments vividly, and encourage player creativity.\n"
    "  - Player Agency: Offer meaningful choices if players ask for choices, avoid railroading, and respect declared player actions.\n"
    "  - Transparency: Never reveal secret rolls, hidden stats, or behind-the-screen logic; use private mechanics discreetly.\n"
    "  - Rulings vs. Rules: Apply rules fairly; when ambiguity arises, make a ruling consistent with core rules, but acknowledge it's a DM's call.\n"
    "  - Consistency: Keep rulings and NPC behavior coherent; avoid contradictory decisions that break immersion.\n"
    "  - Language: Use descriptive, evocative language; avoid technical rule jargon in player-facing narrations.\n"
    "  - Metagame Awareness: Don't leverage player knowledge of rules or story beyond their character's knowledge.\n"
    "  - Respect and Safety: Maintain a respectful tone; refrain from belittling or punishing players arbitrarily; uphold table safety and consent.\n"
    "\n"
    "Important additional instructions:\n"
    "  - If the DM response is irrelevant to the player prompt or the context, rate it Very Bad.\n"
    "  - If the DM response severely misuses, breaks, or abuses D&D rules, rate it Very Bad.\n"
    "  - If the DM response is too overly wordy, rate it Bad or Neutral.\n"
    "\n"
    "<|eot_id|>\n"
    "{player_prompt[i]}\n"
    "<|start_header_id|>DM response<|end_header_id|>\n"
    "{dm_response[i]}\n"
    "<|eot_id|>\n"
    "<|start_header_id|>The DM Judge<|end_header_id|>\n"
    "According to players prompt, I think the DM response is "
    ) for i in range(len(player_prompt))]

    inp = self.tokenizer(
            rewardPrompt, return_tensors='pt',padding=True, truncation=True, max_length=1024
        )#.input_ids.to(self.device)
    input_ids = inp.input_ids.to(self.device)
    mask = inp.attention_mask.to(self.device)
    out = self.model.generate(
        input_ids,
        attention_mask=mask,
        max_new_tokens=4,
        do_sample = False,
        eos_token_id=self.tokenizer.eos_token_id,
        pad_token_id=self.tokenizer.eos_token_id,
    )
    output = []
    score = [-1.,-.75,-.5,-.25,-0.]
    valid_ratings = ["Very Bad", "Bad", "Neutral", "Good", "Very Good"]
    out = tokenizer.batch_decode(out, skip_special_tokens=True)
    for i in out:
      decoded = i.split("According to players prompt, I think the DM response is ")[-1].strip()
      scores = {
          r: difflib.SequenceMatcher(None, decoded.lower(), r.lower()).ratio()
          for r in valid_ratings
      }
      best, best_score = max(scores.items(), key=lambda kv: kv[1])
      output.append(score[valid_ratings.index(best)] if best_score > 0.3 else score[valid_ratings.index("Neutral")])
    return torch.tensor(output)

In [ ]:
voc = tokenizer.vocab
rev = {voc[i]:i for i in voc}
voc['<|eot_id|>']

128009

In [ ]:
rev[128001]

'<|end_of_text|>'

In [ ]:
for param in model.parameters():
  param.requires_grad = False

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [ ]:
from torch.utils.data import Dataset

class DMNSPDataset(Dataset):
    def __init__(self, before_utterances, dm, tokenizer, max_length=1024):
        self.before_utterances = before_utterances
        self.dm = dm
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.promptBegin = (
            "<|begin_of_text|>\n"
            "<|start_header_id|>system<|end_header_id|>\n"
            "You are a highly skilled Dungeon Master (DM) for Dungeons & Dragons 5th Edition. "
            "Your job is to read the player’s input and reply with an immersive, clear, and engaging narration that advances the story and game mechanics.\n"
            "\n"
            "Advice for the DM:\n"
            "  - Describe scenes with vivid sensory details and atmosphere.\n"
            "  - Offer meaningful choices if players ask for choices; respect player agency and avoid railroading.\n"
            "  - Keep secret die rolls hidden; reveal only the results and their effects.\n"
            "  - Use rules faithfully but prioritize fun and pacing over strict book-keeping.\n"
            "  - Improvise when players surprise you, but maintain internal consistency.\n"
            "  - Avoid technical jargon in your narration—stay in character and tone.\n"
            "\n"
            "Basic D&D 5e Rules Summary:\n"
            "  - Initiative: roll a d20 + Dexterity modifier to determine turn order.\n"
            "  - Actions: on your turn you can take one Action, one Bonus Action (if available), and move up to your speed.\n"
            "  - Reactions: special actions triggered outside your turn, such as Opportunity Attacks.\n"
            "  - Advantage & Disadvantage: roll two d20s; take the higher roll for advantage or the lower for disadvantage.\n"
            "  - Spellcasting: consumes spell slots; cantrips are cast at will; concentration holds one spell at a time.\n"
            "<|eot_id|>\n"
        )
        self.promptEnd = (
        "<|start_header_id|>DM response<|end_header_id|>\n"
        "According to players prompt, as a DM of this game I would say "
        )

    def __len__(self):
        return len(self.dm)

    def convert_to_prompt(self,inp):
      out = []
      for line in inp.split('\n'):
        s = line.split('>')
        out.append(f"<|start_header_id|>{s[0][1:]}<|end_header_id|>\n{s[1]}\n <|eot_id|> \n")
      return ''.join(out)

    def __getitem__(self, idx):
        middle = self.convert_to_prompt(self.before_utterances[idx])
        return {
            "input": self.promptBegin+middle+self.promptEnd,
            "playerPrompt": middle
        }
        # return self.promptBegin+middle+self.promptEnd


In [ ]:
train_dataset = DMNSPDataset(train_df['before_utterances'].to_list(), train_df['dm'].to_list(), tokenizer)
val_dataset = DMNSPDataset(val_df['before_utterances'].to_list(), val_df['dm'].to_list(), tokenizer)
test_dataset = DMNSPDataset(test_df['before_utterances'].to_list(), test_df['dm'].to_list(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [ ]:
inp = next(iter(train_dataset))

In [ ]:
print(inp['input'])

<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a highly skilled Dungeon Master (DM) for Dungeons & Dragons 5th Edition. Your job is to read the player’s input and reply with an immersive, clear, and engaging narration that advances the story and game mechanics.

Advice for the DM:
  - Describe scenes with vivid sensory details and atmosphere.
  - Offer meaningful choices if players ask for choices; respect player agency and avoid railroading.
  - Keep secret die rolls hidden; reveal only the results and their effects.
  - Use rules faithfully but prioritize fun and pacing over strict book-keeping.
  - Improvise when players surprise you, but maintain internal consistency.
  - Avoid technical jargon in your narration—stay in character and tone.

Basic D&D 5e Rules Summary:
  - Initiative: roll a d20 + Dexterity modifier to determine turn order.
  - Actions: on your turn you can take one Action, one Bonus Action (if available), and move up to your speed.
  - Reactio

# Lovely Training loop & DARK MAGIC

In [ ]:
def print_trainable_parameters(model):
    """
    printing the number of trainable paramters in the model
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

In [ ]:
# padding='max_length',
# max_length=self.max_length
class PPO_handler():
  def __init__(self,model):
    self.Teacher = TeacherModel()
    self.model = model
  def computeLoss(self,inputs,attMap,prompt):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    inputs = torch.tensor(inputs).to(device)
    attMap = torch.tensor(attMap).to(device)
    outputs = self.model.generate(input_ids=inputs, attention_mask=attMap, max_new_tokens=100, do_sample=False,eos_token_id=tokenizer.eos_token_id,pad_token_id=tokenizer.eos_token_id,)[:, inputs.shape[1]:]
    out = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    rewards = self.Teacher.score(prompt,out).to(inputs.device)
    X = torch.cat([inputs, outputs[:, :-1]], dim=1)
    Y = torch.cat([inputs[:, 1:], outputs], dim=1)
    labels_mask = torch.cat([torch.zeros_like(inputs),torch.ones((inputs.shape[0], outputs.shape[1]-1)).to(inputs.device)], dim=1)
    labels_mask = labels_mask * (Y != 128001).int()
    R = (torch.ones_like(Y).to(inputs.device) * rewards.unsqueeze(dim=1)) * labels_mask
    output = model(X.cuda())
    logprobs = output.logits.cpu().log_softmax(dim=2).to(inputs.device)
    P_y_x = logprobs.gather(2, Y.unsqueeze(2)).squeeze()
    loss = (P_y_x * R).mean()
    return loss

In [ ]:
config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    target_modules = ["k_proj", "v_proj"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

fabric = Fabric(accelerator="cuda", devices=1, precision="bf16")
device = fabric.device
fabric.launch()
torch.set_float32_matmul_precision("medium")

INFO: Using bfloat16 Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)


trainable params: 5242880 || all params: 4545843200 || trainable%: 0.11533349852454215


INFO: You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


In [ ]:
from peft import LoraConfig, get_peft_model, PeftModelForCausalLM
###########################################################################
#                          WE HAVE TO TUNE THIS                           #
###########################################################################

config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    target_modules = ["k_proj", "v_proj"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

fabric = Fabric(accelerator="cuda", devices=1, precision="bf16")
device = fabric.device
fabric.launch()
torch.set_float32_matmul_precision("medium")

class Trainer(): ### SET CONFIG HERE
  def __init__(self,model):
    self.model = model.to(device)
    self.loss = nn.CrossEntropyLoss().to(device)
    self.optimizer = AdamW(self.model.parameters(), lr=1e-5, weight_decay=0.01)
    self.wait = 5 # FOR EARLY STOPPING
    self.currentEpoch = 0
    self.steps = 5
    self.batch = 8
    self.best = 2e9
    self.earlyBest = 2e9
    self.counter = 0
    self.trainHistory = []
    self.score = []
    self.train_loader = iter(DataLoader(train_dataset, batch_size=self.batch, shuffle=True))
    self.valid_loader = iter(DataLoader(val_dataset, batch_size=self.batch, shuffle=True))
    self.util = PPO_handler(lora_model)

###########################################################################

  def TrainingLoop(self):
    self.model.train()
    score = []
    progress_bar = tqdm(range(self.steps), leave=False)
    for _ in progress_bar:
      try:
        batch = next(self.train_loader)
      except:
        self.train_loader = iter(DataLoader(train_dataset, batch_size=self.batch, shuffle=True))
        batch = next(self.train_loader)
      # input_ids = batch['input_ids'].to(device)
      # attention_mask = batch['attention_mask'].to(device)
      # index = batch['index'].to(device)
      inp = tokenizer(batch['input'], return_tensors='pt',padding=True, truncation=True, max_length=1024)
      self.optimizer.zero_grad()
      loss = self.util.computeLoss(inp['input_ids'],inp['attention_mask'],batch['playerPrompt'])
      loss.backward()
      self.optimizer.step()
      score.append(loss.cpu().detach().numpy())
    progress_bar.close()
    self.score = score
    return np.mean([i for i in score])

  def Train(self,epoch,save_best=False,restore_best = False,early = False):
    path = "weight/"
    for i in range(self.currentEpoch,epoch):
      print("Epoch : ",i+1)
      train_loss = self.TrainingLoop()
      gc.collect()
      self.trainHistory.append(train_loss)
      print("PPO Loss : ",train_loss)
      if(device == 'cuda'):
        torch.cuda.empty_cache()
      gc.collect()
      if(save_best):
        if(train_loss<self.best):
          self.best = train_loss
          os.makedirs("weight/bestRL", exist_ok=True)
          self.model.save_pretrained(path+"bestRL",safe_serialization=True)
          torch.save(self.optimizer.state_dict(), "optimizer")
      if(early):
        if(train_loss<self.earlyBest):
          print("Improved present loss value :",train_loss)
          self.earlyBest = train_loss
          self.counter = 0
        else:
          print("Not improved current val :",self.earlyBest)
          if(self.counter > self.wait):
            break
          else:
            self.counter += 1
      if(restore_best):
        if "best" in os.listdir(path):
          self.model= PeftModelForCausalLM.from_pretrained(self.model, path+"bestRL")
      self.log()

  def log(self):
    path = "logger.csv"
    df = pd.DataFrame(columns = ["Epoch","Train Loss"])
    for i in range(len(self.trainHistory)):
      df.loc[i] = [i+1,self.trainHistory[i]]
    df.to_csv(path,index=False)

  def resumeTraining(self):
    path = "weight/"
    df = pd.read_csv('logger.csv')
    self.model= PeftModelForCausalLM.from_pretrained(self.model, path+"bestRL")
    self.optimizer.load_state_dict(torch.load("optimizer"))
    self.currentEpoch = int(df['Epoch'].max())
    self.trainHistory = [df['Train Loss'][i] for i in range(len(df))]
    self.earlyBest = df['Train Loss'].min()
    self.best = self.earlyBest
    for index,row in df.iterrows():
      print("Epoch : "+str(row['Epoch']))
      print(" Train Loss : "+str(row['Train Loss']))

INFO: Using bfloat16 Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)


trainable params: 5242880 || all params: 4545843200 || trainable%: 0.11533349852454215


In [ ]:
trainer = Trainer(lora_model)
# trainer.resumeTraining()
trainer.Train(100,save_best=True,restore_best = False,early = True) #restore_best is restore best weight if loss increases compared to loss of the best weight

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch :  1


  0%|          | 0/5 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


...

PPO Loss :  0.01608643
Not improved current val : 0.014242267
Epoch :  67


  0%|          | 0/5 [00:00<?, ?it/s]

PPO Loss :  0.015397169
Not improved current val : 0.014242267
Epoch :  68


  0%|          | 0/5 [00:00<?, ?it/s]

PPO Loss :  0.015226667
Not improved current val : 0.014242267


# Inference

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Automatically map layers across GPUs if available
    load_in_4bit=True,   # Enables memory-efficient training
    torch_dtype=torch.bfloat16
)
model = PeftModelForCausalLM.from_pretrained(model, 'weight/'+"bestRL")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
vanila = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Automatically map layers across GPUs if available
    load_in_4bit=True,   # Enables memory-efficient training
    torch_dtype=torch.bfloat16
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
index = 37700
inp = train_dataset[index]['input']
print("INPUT")
print(inp)

INPUT
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a highly skilled Dungeon Master (DM) for Dungeons & Dragons 5th Edition. Your job is to read the player’s input and reply with an immersive, clear, and engaging narration that advances the story and game mechanics.

Advice for the DM:
  - Describe scenes with vivid sensory details and atmosphere.
  - Offer meaningful choices if players ask for choices; respect player agency and avoid railroading.
  - Keep secret die rolls hidden; reveal only the results and their effects.
  - Use rules faithfully but prioritize fun and pacing over strict book-keeping.
  - Improvise when players surprise you, but maintain internal consistency.
  - Avoid technical jargon in your narration—stay in character and tone.

Basic D&D 5e Rules Summary:
  - Initiative: roll a d20 + Dexterity modifier to determine turn order.
  - Actions: on your turn you can take one Action, one Bonus Action (if available), and move up to your speed.
  - R

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tmp = tokenizer(inp, return_tensors='pt',padding=True, truncation=True, max_length=1024)
inputs = torch.tensor(tmp['input_ids']).to(device)
attMap = torch.tensor(tmp['attention_mask']).to(device)
outputs = model.generate(input_ids=inputs, attention_mask=attMap, max_new_tokens=200, do_sample=False,eos_token_id=tokenizer.eos_token_id,pad_token_id=tokenizer.eos_token_id,)[:, inputs.shape[1]:]
out = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print("RL VERSION")
print()
print('\n'.join(out[0].split('.')))
print()
print("VANILA VERSION")
outputs = vanila.generate(input_ids=inputs, attention_mask=attMap, max_new_tokens=200, do_sample=False,eos_token_id=tokenizer.eos_token_id,pad_token_id=tokenizer.eos_token_id,)[:, inputs.shape[1]:]
out = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('\n'.join(out[0].split('.')))

RL VERSION

 that you manage to grab a few of your belongings, including your backpack, which contains some useful items like a water bottle, a first-aid kit, and a small pouch of gold coins
 You also retrieve your trusty dagger, which you sheathe at your belt
 As you start to put yourself together, you notice that the room is still in disarray, with furniture overturned and debris scattered everywhere
 The air is thick with the smell of smoke and sweat


You take a deep breath, trying to steady yourself, and begin to yell "Bloody murder!" at the top of your lungs
 Your voice echoes through the empty halls, but there's no response
 The silence is oppressive, and you can't shake the feeling that you're being watched
 You look around, trying to get your bearings, and notice that the door to the bedroom is still open, with a sliver of moonlight streaming in
 You also notice that the window is broken, and the wind is howling

VANILA VERSION
 You manage to grab a few of your belongings, a s

# Stop notebook

In [ ]:
from google.colab import runtime
runtime.unassign()